In [ ]:
# 导包
import torch
import os
import argparse
import pandas as pd
import tqdm
import numpy as np
from torch.nn.functional import avg_pool2d

# os.environ['NUMEXPR_MAX_THREADS'] = '48'

# 导入各个指标
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')

# task select
from target_model.task_select import get_dataloader_and_model,get_dataloader,get_models,get_infotopo_para

# utils
from ppsplit.utils import concat_weights, create_dir, load_json, save_json

# defense:
from ppsplit.defense.obfuscation.scheduler import Scheduler
from ppsplit.defense.dp.posthoc.posthoc import PosthocDefense

/home/dengruijun/miniconda3/envs/drj-pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Obfuscation

In [ ]:
config = load_json('./config/nopeek.json')
# config = load_json('./config/shredder.json')
# config = load_json('./config/cloak.json')
# config = load_json('./config/uniform_noise.json')

print(config)
args = config['general']
config['defense']['device']=args['device']

{'defense': {'method': 'uniform_noise', 'client': {'model_name': 'resnet18', 'split_layer': 6, 'pretrained': False, 'optimizer': 'adam', 'lr': 0.0003, 'distribution': 'gaussian', 'mean': 0, 'sigma': 1}, 'server': {'model_name': 'resnet18', 'split_layer': 6, 'logits': 2, 'pretrained': False, 'lr': 0.0003, 'optimizer': 'adam'}, 'learning_rate': 0.01, 'total_epochs': 150, 'training_batch_size': 128, 'dataset': 'fairface', 'protected_attribute': 'data', 'prediction_attribute': 'gender', 'img_size': 128, 'split': False, 'test_batch_size': 64, 'exp_id': '1', 'exp_keys': ['client.distribution', 'client.mean', 'client.sigma'], 'device': 'cuda:0'}, 'general': {'result_dir': '20241228-defense/', 'test_num': 'uniform_noise', 'device': 'cuda:0', 'dataset': 'CIFAR10', 'oneData_bs': 1, 'train_bs': 128, 'test_bs': 64, 'model': 'VGG5', 'split_layer': 2, 'ep': -1, 'no_dense': 0, 'noise_scale': 0}}


In [ ]:
# %%
data_msg = get_dataloader(args)
model_msg = get_models(args)
msg = {**model_msg,**data_msg}

# 数据集
one_data_loader,trainloader,testloader = data_msg['one_data_loader'],data_msg['trainloader'], data_msg['testloader']
data_interval = data_msg['data_interval']
data_type = msg['data_type']

# 模型
client_net = model_msg['client_net']
server_net,unit_net = model_msg['server_net'], model_msg['unit_net']
image_deprocess = model_msg['image_deprocess']

# 路径
results_dir = model_msg['results_dir']
inverse_dir = results_dir + 'layer' + str(args['split_layer'])+'/'
# data_type = 1 if args['dataset'] == 'CIFAR10' else 0
split_layer = args['split_layer']

print('results_dir:', results_dir)

create_dir(inverse_dir)

# net使用
client_net = client_net.to(args['device'])
server_net = server_net.to(args['device'])
unit_net = unit_net.to(args['device'])


unit_net weights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked', 'features.8.weight', 'features.8.bias', 'features.9.weight', 'features.9.bias', 'features.9.running_mean', 'features.9.running_var', 'features.9.num_batches_tracked', 'denses.0.weight', 'denses.0.bias', 'denses.1.weight', 'denses.1.bias'])
client_net cweights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked'])
len unit_net weights:  25


KeyError: 'no_pool'

In [ ]:
# 防御
config['defense']["results_dir"] = results_dir
# config["1"] = results_dir
defense_scheduler = Scheduler(config)
if config['defense']['method']=='cloak':
    client_net = None
    server_net = unit_net

# 初始化和run存储模型
defense_scheduler.initialize(train_loader = trainloader, test_loader = testloader, client_model = client_net, server_model = server_net)
client_net,server_net = defense_scheduler.run_job()

# 拼接模型并保存
if client_net: # 如果有client_net，则拼接
    new_weights_unit = concat_weights(unit_net.state_dict(),client_net.state_dict(),server_net.state_dict())
else: # 如果没有client_net，则直接保存server_net
    new_weights_unit = server_net.state_dict()

unit_net.load_state_dict(new_weights_unit)
torch.save(unit_net.state_dict(), inverse_dir + 'unit_net_defensed.pth')

print("model saved in ",inverse_dir + 'unit_net_defensed.pth')

# 2. DP

In [4]:
config = load_json('./config/posthoc.json')

args =  {
        "result_dir":"20241228-defense/",
        "test_num":"Posthoc",
        "device":"cuda:0",

        "dataset":"CIFAR10",
        "oneData_bs": 1,
        "train_bs": 32,
        "test_bs": 64,
        
        "model":"VGG5",
        "split_layer": 2,
        "ep":-1,
        "no_dense": 0,
        
        "noise_scale": 0
    }
print(config)
print(args)

{'arl_config': {'alpha': 0.99, 'dset': 'utkface', 'noise_reg': 1, 'sigma': 0.01, 'siamese_reg': 1, 'margin': 25, 'lambda': 1.0, 'tag': 'gender', 'device': 'cuda:0', 'epoch': 50}, 'eval_config': {'epsilon': 5, 'delta': 0.1, 'radius': 0.2, 'eval_size': 100, 'proposed_bound': 0.84, 'max_upper_bound': 2}}
{'result_dir': '20241228-defense/', 'test_num': 'Posthoc', 'device': 'cuda:0', 'dataset': 'CIFAR10', 'oneData_bs': 1, 'train_bs': 32, 'test_bs': 64, 'model': 'VGG5', 'split_layer': 2, 'ep': -1, 'no_dense': 0, 'noise_scale': 0}


In [5]:
data_msg = get_dataloader(args)
model_msg = get_models(args)
# infotopo_msg = get_infotopo_para(args)
msg = {**model_msg,**data_msg}

# 数据集
one_data_loader,trainloader,testloader = data_msg['one_data_loader'],data_msg['trainloader'], data_msg['testloader']

# 模型
client_net = model_msg['client_net']
client_net,decoder_net = model_msg['client_net'],model_msg['decoder_net']
decoder_route = model_msg['decoder_route']
server_net = model_msg['server_net']

image_deprocess = model_msg['image_deprocess']

# 路径
results_dir = model_msg['results_dir']
split_layer = args['split_layer']

print('results_dir:', results_dir)
print('decoder_route:', decoder_route)

create_dir(results_dir)

# decoder_route = "/home/dengruijun/data/FinTech/PP-Split/results/inverse-model-results-20240414/VGG5/2/Decoder-layer2.pth"
# # decoder_route = "/home/dengruijun/data/FinTech/PP-Split/results/20241228-defense/VGG5/Posthoc/_noisereg_0.01_siamesereg_25_1.0/adv.pth"
# if os.path.isfile(decoder_route): # 如果已经训练好了
#     print("=> loading decoder model '{}'".format(decoder_route))
#     decoder_net = torch.load(decoder_route)
# else:
#     print("=> no decoder model found at '{}'".format(decoder_route))

unit_net weights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked', 'features.8.weight', 'features.8.bias', 'features.9.weight', 'features.9.bias', 'features.9.running_mean', 'features.9.running_var', 'features.9.num_batches_tracked', 'denses.0.weight', 'denses.0.bias', 'denses.1.weight', 'denses.1.bias'])
client_net cweights:  odict_keys(['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.1.running_mean', 'features.1.running_var', 'features.1.num_batches_tracked', 'features.4.weight', 'features.4.bias', 'features.5.weight', 'features.5.bias', 'features.5.running_mean', 'features.5.running_var', 'features.5.num_batches_tracked'])
len unit_net weights:  25


In [6]:
# 加载 posthoc 防御方法
# 用于testloader的batch size
posthoc_defense = PosthocDefense(config, 
                                 train_loader=trainloader, 
                                 test_loader=testloader,
                                 client_net=client_net, 
                                 server_net=server_net, 
                                 decoder_net=decoder_net,
                                 results_dir=results_dir,
                                 device=args['device'])

2025-02-23 01:15:29,531 - ppsplit.defense.dp.posthoc.posthoc - INFO - {'alpha': 0.99, 'dset': 'utkface', 'noise_reg': 1, 'sigma': 0.01, 'siamese_reg': 1, 'margin': 25, 'lambda': 1.0, 'tag': 'gender', 'device': 'cuda:0', 'epoch': 50}
2025-02-23 01:15:29,533 - ppsplit.defense.dp.posthoc.posthoc - INFO - {'epsilon': 5, 'delta': 0.1, 'radius': 0.2, 'eval_size': 100, 'proposed_bound': 0.84, 'max_upper_bound': 2}


device: cuda:0
ARL base_dir: ../../results/20241228-defense//VGG5/Posthoc/_noisereg_0.01_siamesereg_25_1.0


In [8]:
# 加载decoder模型
if not os.path.isfile(decoder_route): # 如果没有训练decoder
    # 训练decoder
    args['train_bs']=32
    args['test_bs']=32
    msg_data = get_dataloader(args)

    posthoc_defense.arl_obj.setup_path()
    print("starting training ARL")
    
    posthoc_defense.arl_obj.train()
    
else:
    print("Load decoder model from:",decoder_route)

print(decoder_net)

Load decoder model from: /home/dengruijun/data/FinTech/PP-Split/results/inverse-model-results-20240414/VGG5/2/Decoder-layer2.pth
VGG5Decoder(
  (features): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (4): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
  )
  (denses): Sequential()
)


In [ ]:
# ARL训练
# 用于train_loader的batch size
args['train_bs']=32
args['test_bs']=32
msg_data = get_dataloader(args)
posthoc_defense.train_ARL(train_loader=trainloader, test_loader=testloader)

In [9]:
# posthoc 框架训练
posthoc_defense.arl_obj.on_cpu() # move the models to cpu
posthoc_defense.defense_train()

AttributeError: 'VGG' object has no attribute 'layer_sizes'

In [ ]:
# posthoc 隐私测试
posthoc_defense.defense_test()